In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [3]:
portfolio = pd.read_excel(io="Workshop Data.xlsx",sheet_name="Portfolio")
TenYearYield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="10yUST Yields")
HYyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="HY Index")
IGyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="IG Index")
MoveIdx = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="MOVE Index")
Vix = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="VIX")
SPY_Idx = pd.read_excel(io="SP500 Index data.xlsx",sheet_name="SPY")

portfolio = portfolio.set_index("Date")
TenYearYield = TenYearYield.set_index("Date").sort_index()
HYyield = HYyield.set_index("Date").sort_index()
IGyield = IGyield.set_index("Date").sort_index()
MoveIdx = MoveIdx.set_index("Date").sort_index()
Vix = Vix.set_index("Date").sort_index()
SPY_Idx = SPY_Idx.set_index("Date").sort_index()

portfolio.loc[:,"LQDCumDiv"]=portfolio.loc[:,"LQD Dividends"][::-1].cumsum()[::-1]
portfolio.loc[:,"TotalValueLQD"]=portfolio.loc[:,"LQD Position"]+portfolio.loc[:,"LQDCumDiv"]
portfolio = portfolio.sort_index()

LQDCorrData = pd.DataFrame(portfolio.loc["2021-01-11":"2025-12-31","TotalValueLQD"]).merge(HYyield.loc["2021-01-11":"2025-12-31","OAS_SOVEREIGN_CURVE"],left_index=True,right_index=True)
LQDCorrData.loc[:,"TotalReturnsLQD"] = LQDCorrData["TotalValueLQD"].pct_change()
LQDCorrData = LQDCorrData.loc["2021-01-11":"2025-12-31"].merge(HYyield.loc["2021-01-11":"2025-12-31","INDEX_Z_SPREAD_BP"],left_index=True,right_index=True)
LQDCorrData = LQDCorrData.rename(columns={"OAS_SOVEREIGN_CURVE":"HY_Index_OAS","INDEX_Z_SPREAD_BP":"HY_INDEX_Z_SPREAD"})
LQDCorrData.loc[:,"HY_Index_OAS_Diff"]=LQDCorrData["HY_Index_OAS"].diff()
LQDCorrData.loc[:,"HY_INDEX_Z_SPREAD_Diff"]=LQDCorrData["HY_INDEX_Z_SPREAD"].diff()

LQDCorrData = LQDCorrData.loc["2021-01-11":"2025-12-31"].merge(TenYearYield.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
LQDCorrData = LQDCorrData.rename(columns={"PX_LAST":"10y_UST_INDEX_PX"})
LQDCorrData.loc[:,"10yYieldDiff"] = LQDCorrData["10y_UST_INDEX_PX"].diff()

LQDCorrData = LQDCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31","OAS_SOVEREIGN_CURVE"],left_index=True,right_index=True)
LQDCorrData = LQDCorrData.rename(columns={"OAS_SOVEREIGN_CURVE":"IG_Index_OAS"})
LQDCorrData.loc[:,"IG_Index_OAS_Diff"]=LQDCorrData["IG_Index_OAS"].diff()
LQDCorrData = LQDCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31","INDEX_Z_SPREAD_BP"],left_index=True,right_index=True)
LQDCorrData = LQDCorrData.rename(columns={"INDEX_Z_SPREAD_BP":"IG_INDEX_Z_SPREAD"})
LQDCorrData.loc[:,"IG_Index_Z_SPREAD_Diff"]=LQDCorrData["IG_INDEX_Z_SPREAD"].diff()

LQDCorrData = LQDCorrData.loc["2021-01-11":"2025-12-31"].merge(SPY_Idx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
LQDCorrData = LQDCorrData.rename(columns={"PX_LAST":"SPY_Idx_LAST_PX"})
LQDCorrData.loc[:,"ReturnSPY_Idx"] = LQDCorrData["SPY_Idx_LAST_PX"].pct_change()

LQDCorrData = LQDCorrData.loc["2021-01-11":"2025-12-31"].merge(MoveIdx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
LQDCorrData = LQDCorrData.rename(columns={"PX_LAST":"MOVE Idx"})

LQDCorrData = LQDCorrData.loc["2021-01-11":"2025-12-31"].merge(Vix.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
LQDCorrData = LQDCorrData.rename(columns={"PX_LAST":"Vix Idx"})

LQDCorrData=LQDCorrData.dropna()
LQDCorrData = LQDCorrData.sort_index(ascending=False)

In [16]:
LQDCorrData[["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_OAS_Diff","ReturnSPY_Idx","MOVE Idx"]]

,HY_Index_OAS_Diff,10yYieldDiff,IG_Index_OAS_Diff,ReturnSPY_Idx,MOVE Idx
Date,,,,,
2025-12-31,-0.010172,0.0451,0.005634,-0.007358,63.96
2025-12-30,-0.037478,0.0117,-0.003152,-0.001376,63.98
2025-12-29,0.001774,-0.0175,0.007974,-0.003492,59.21
2025-12-26,0.037774,-0.0058,-0.003501,-0.000304,58.50
2025-12-24,-0.007383,-0.0293,-0.006932,0.003222,59.48
...,...,...,...,...,...
2021-01-19,-0.021835,0.0051,-0.013045,0.008136,45.23
2021-01-15,0.025426,-0.0457,0.008244,-0.007190,45.14
2021-01-14,-0.069708,0.0460,-0.005403,-0.003753,46.23


In [17]:
LQDCorrData[["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_OAS_Diff","ReturnSPY_Idx","MOVE Idx"]].corr()

,HY_Index_OAS_Diff,10yYieldDiff,IG_Index_OAS_Diff,ReturnSPY_Idx,MOVE Idx
HY_Index_OAS_Diff,1.000000,-0.269423,0.751647,-0.576388,0.053103
10yYieldDiff,-0.269423,1.000000,-0.190721,-0.066743,0.019557
IG_Index_OAS_Diff,0.751647,-0.190721,1.000000,-0.518336,0.071133
ReturnSPY_Idx,-0.576388,-0.066743,-0.518336,1.000000,-0.038251
MOVE Idx,0.053103,0.019557,0.071133,-0.038251,1.000000


In [10]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(LQDCorrData[["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_OAS_Diff","ReturnSPY_Idx","MOVE Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_OAS_Diff","ReturnSPY_Idx","MOVE Idx"]))

X_pca = pcav1.transform(X_Std)

LQDCorrData.loc[:,"PC1"]=X_pca[:,0]
LQDCorrData.loc[:,"PC2"]=X_pca[:,1]
LQDCorrData.loc[:,"PC3"]=X_pca[:,2]
LQDCorrData.loc[:,"PC4"]=X_pca[:,3]
LQDCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(LQDCorrData[["PC1","PC2","PC3","PC4"]])
Y = LQDCorrData["TotalReturnsLQD"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.45782498 0.21282642 0.19641817 0.08654985 0.04638058]
   HY_Index_OAS_Diff  10yYieldDiff  IG_Index_OAS_Diff  ReturnSPY_Idx  MOVE Idx
0           0.606625     -0.186560           0.585843      -0.499187  0.069297
1          -0.074357      0.839844          -0.010209      -0.360899  0.398473
2          -0.010135     -0.333961          -0.005056       0.233329  0.913181
3           0.178136      0.358583           0.555279       0.727247 -0.049631
4           0.771130      0.140520          -0.590189       0.192963  0.007376


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsLQD   R-squared:                       0.871
Model:                            OLS   Adj. R-squared:                  0.870
Method:                 Least Squares   F-statistic:                     2078.
Date:                Thu, 15 Jan 2026   Prob (F-statistic):               0.00
Time:                        19:29:09   Log-Likelihood:                 6039.8
No. Observations:                1240   AIC:                        -1.207e+04
Df Residuals:                    1235   BIC:                        -1.204e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.921e-05   5.28e-05     -0.364      0.716      -0.000    8.44e-05
PC1           -0.0003   3.49e-05     -8.310      0.000      -0.000      -0.000
PC2           -0.0042   5.12e-05    -81.277      0.000      -0.004      -0.004
PC3            0.0018   5.33e-05     34.376      0.000       0.002       0.002
PC4           -0.0017   8.02e-05    -21.357      0.000      -0.002      -0.002
==============================================================================
Omnibus:                       69.616   Durbin-Watson:                   2.449
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              156.553
Skew:                          -0.336   Prob(JB):                     1.01e-34
Kurtosis:                       4.606   Cond. No.                         2.30
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<u> Or this one to reach close to 87% </u>

See the other one indicated below first

In [12]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(LQDCorrData[["10yYieldDiff","IG_Index_OAS_Diff","ReturnSPY_Idx","MOVE Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["10yYieldDiff","IG_Index_OAS_Diff","ReturnSPY_Idx","MOVE Idx"]))

X_pca = pcav1.transform(X_Std)

LQDCorrData.loc[:,"PC1"]=X_pca[:,0]
LQDCorrData.loc[:,"PC2"]=X_pca[:,1]
LQDCorrData.loc[:,"PC3"]=X_pca[:,2]
LQDCorrData.loc[:,"PC4"]=X_pca[:,3]


X = sm.add_constant(LQDCorrData[["PC1","PC2","PC3","PC4"]])
Y = LQDCorrData["TotalReturnsLQD"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.38590336 0.26315094 0.24547156 0.10547414]
   10yYieldDiff  IG_Index_OAS_Diff  ReturnSPY_Idx  MOVE Idx
0     -0.163054           0.712637      -0.668942  0.134454
1      0.884970          -0.114220      -0.263756  0.366355
2     -0.311477          -0.019339       0.240080  0.919222
3      0.305326           0.691902       0.652159 -0.052313


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsLQD   R-squared:                       0.874
Model:                            OLS   Adj. R-squared:                  0.874
Method:                 Least Squares   F-statistic:                     2149.
Date:                Thu, 15 Jan 2026   Prob (F-statistic):               0.00
Time:                        19:29:41   Log-Likelihood:                 6057.9
No. Observations:                1240   AIC:                        -1.211e+04
Df Residuals:                    1235   BIC:                        -1.208e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.921e-05    5.2e-05     -0.369      0.712      -0.000    8.29e-05
PC1           -0.0008   4.19e-05    -18.038      0.000      -0.001      -0.001
PC2           -0.0041   5.07e-05    -81.575      0.000      -0.004      -0.004
PC3            0.0018   5.25e-05     33.413      0.000       0.002       0.002
PC4           -0.0018   8.01e-05    -22.339      0.000      -0.002      -0.002
==============================================================================
Omnibus:                       65.490   Durbin-Watson:                   2.462
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              144.661
Skew:                          -0.320   Prob(JB):                     3.87e-32
Kurtosis:                       4.546   Cond. No.                         1.91
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(LQDCorrData[["HY_Index_OAS_Diff","10yYieldDiff","ReturnSPY_Idx","MOVE Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["HY_Index_OAS_Diff","10yYieldDiff","ReturnSPY_Idx","MOVE Idx"]))

X_pca = pcav1.transform(X_Std)

LQDCorrData.loc[:,"PC1"]=X_pca[:,0]
LQDCorrData.loc[:,"PC2"]=X_pca[:,1]
LQDCorrData.loc[:,"PC3"]=X_pca[:,2]
LQDCorrData.loc[:,"PC4"]=X_pca[:,3]

X = sm.add_constant(LQDCorrData[["PC1","PC2","PC3","PC4"]])
Y = LQDCorrData["TotalReturnsLQD"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.40455834 0.26599148 0.24551432 0.08393585]
   HY_Index_OAS_Diff  10yYieldDiff  ReturnSPY_Idx  MOVE Idx
0           0.716722     -0.239390      -0.648188  0.094095
1          -0.085403      0.844974      -0.348994  0.396141
2          -0.014831     -0.331027       0.238378  0.912895
3           0.691951      0.345154       0.633428 -0.029003


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsLQD   R-squared:                       0.836
Model:                            OLS   Adj. R-squared:                  0.835
Method:                 Least Squares   F-statistic:                     1572.
Date:                Thu, 15 Jan 2026   Prob (F-statistic):               0.00
Time:                        19:29:56   Log-Likelihood:                 5892.2
No. Observations:                1240   AIC:                        -1.177e+04
Df Residuals:                    1235   BIC:                        -1.175e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.921e-05   5.95e-05     -0.323      0.747      -0.000    9.75e-05
PC1           -0.0001   4.67e-05     -2.568      0.010      -0.000   -2.83e-05
PC2           -0.0042   5.77e-05    -72.302      0.000      -0.004      -0.004
PC3            0.0018      6e-05     30.384      0.000       0.002       0.002
PC4           -0.0012      0.000    -11.492      0.000      -0.001      -0.001
==============================================================================
Omnibus:                       88.386   Durbin-Watson:                   2.246
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              304.999
Skew:                          -0.277   Prob(JB):                     5.89e-67
Kurtosis:                       5.366   Cond. No.                         2.20
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<u> So far I like this one more: </u>

In [14]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(LQDCorrData[["10yYieldDiff","ReturnSPY_Idx","MOVE Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["10yYieldDiff","ReturnSPY_Idx","MOVE Idx"]))

X_pca = pcav1.transform(X_Std)

LQDCorrData.loc[:,"PC1"]=X_pca[:,0]
LQDCorrData.loc[:,"PC2"]=X_pca[:,1]
LQDCorrData.loc[:,"PC3"]=X_pca[:,2]


X = sm.add_constant(LQDCorrData[["PC1","PC2","PC3"]])
Y = LQDCorrData["TotalReturnsLQD"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.36211729 0.3278092  0.31007351]
   10yYieldDiff  ReturnSPY_Idx  MOVE Idx
0     -0.610711       0.663489 -0.432221
1     -0.465865       0.140325  0.873658
2      0.640314       0.734909  0.223398


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsLQD   R-squared:                       0.829
Model:                            OLS   Adj. R-squared:                  0.828
Method:                 Least Squares   F-statistic:                     1995.
Date:                Thu, 15 Jan 2026   Prob (F-statistic):               0.00
Time:                        19:30:12   Log-Likelihood:                 5866.1
No. Observations:                1240   AIC:                        -1.172e+04
Df Residuals:                    1236   BIC:                        -1.170e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.921e-05   6.07e-05     -0.316      0.752      -0.000    9.99e-05
PC1            0.0037   5.82e-05     63.039      0.000       0.004       0.004
PC2            0.0022   6.12e-05     36.722      0.000       0.002       0.002
PC3           -0.0016   6.29e-05    -25.726      0.000      -0.002      -0.001
==============================================================================
Omnibus:                      101.730   Durbin-Watson:                   2.181
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              413.948
Skew:                          -0.282   Prob(JB):                     1.30e-90
Kurtosis:                       5.774   Cond. No.                         1.08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(LQDCorrData[["10yYieldDiff","MOVE Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["10yYieldDiff","MOVE Idx"]))

X_pca = pcav1.transform(X_Std)

LQDCorrData.loc[:,"PC1"]=X_pca[:,0]
LQDCorrData.loc[:,"PC2"]=X_pca[:,1]

X = sm.add_constant(LQDCorrData[["PC1","PC2"]])
Y = LQDCorrData["TotalReturnsLQD"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.50977854 0.49022146]
   10yYieldDiff  MOVE Idx
0      0.707107  0.707107
1      0.707107 -0.707107


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsLQD   R-squared:                       0.738
Model:                            OLS   Adj. R-squared:                  0.737
Method:                 Least Squares   F-statistic:                     1740.
Date:                Thu, 15 Jan 2026   Prob (F-statistic):               0.00
Time:                        19:30:18   Log-Likelihood:                 5601.5
No. Observations:                1240   AIC:                        -1.120e+04
Df Residuals:                    1237   BIC:                        -1.118e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.921e-05   7.51e-05     -0.256      0.798      -0.000       0.000
PC1           -0.0032   7.44e-05    -42.508      0.000      -0.003      -0.003
PC2           -0.0031   7.59e-05    -40.893      0.000      -0.003      -0.003
==============================================================================
Omnibus:                      117.680   Durbin-Watson:                   2.201
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              658.596
Skew:                           0.219   Prob(JB):                    9.72e-144
Kurtosis:                       6.543   Cond. No.                         1.02
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""